In [61]:
#! pip install -r requirements.txt

In [62]:
import pandas as pd
import numpy as np

In [63]:
def remove_unused(df: pd.DataFrame, cols: list) -> pd.DataFrame:
    df_cp = df.copy()
    df_cp = df.drop(cols, axis=1)
    return df_cp
    

In [64]:
def angle_to_coord(row):
    alpha = row['alpha']
    delta = row['delta']
    x = np.cos(alpha) * np.cos(delta)
    y = np.sin(alpha) * np.cos(delta)
    z = np.sin(delta)
    return x, y, z

In [65]:
def spherical_to_castesian(df: pd.DataFrame) -> pd.DataFrame:
    df_unified = df.copy()
    df_unified['xyz'] = df.apply(angle_to_coord, axis=1)
    df_split = df_unified.copy()
    df_split[['x', 'y', 'z']] = pd.DataFrame(df_unified['xyz'].tolist(), index=df.index)
    df_split = remove_unused(df_split, ['xyz'])
    return df_split

In [66]:
df = pd.read_csv('../data/star_classification_10.csv', index_col='obj_ID')
df = remove_unused(df, ['run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'plate', 'MJD', 'fiber_ID'])
df = spherical_to_castesian(df)

In [67]:
df.head()

,alpha,delta,u,g,r,i,z,class,redshift,x,y
obj_ID,,,,,,,,,,,
1.237663e+18,15.342907,0.794882,18.74547,17.49025,16.89122,16.57350,0.713781,GALAXY,0.042002,-0.654218,0.250033
1.237664e+18,120.365538,55.660432,19.99985,19.68133,19.50156,19.17364,-0.775960,QSO,1.633797,0.348727,0.525619
1.237655e+18,245.610038,42.974786,23.11792,20.81292,18.88351,18.12335,-0.845505,GALAXY,0.454852,0.450750,0.286260
1.237660e+18,127.957356,6.647703,21.94454,21.01012,20.93496,20.93184,0.356499,QSO,2.608515,-0.618062,0.700648
1.237665e+18,159.174526,35.881846,18.89945,17.68422,17.02925,16.60000,-0.969780,GALAXY,0.083804,0.122096,-0.211234


In [68]:
df.to_csv('../data/star_classification_10_cartesian.csv')